# Generate C code

In [22]:
import torch.nn as nn
from stable_baselines3 import PPO

# big randomization
# path = 'models/nn_size_comparison/run1_64_64_64/98000000.zip'
# 3inch fixed params
# path = 'models/3inch_drone/run0_fixed_3inch/59000000.zip'
# 3inch 10% randomization
# path = 'models/3inch_drone/run0_3inch_10_percent/55000000.zip'
# 3inch 20% randomization
# path = 'models/3inch_drone/run1_3inch_20_percent/86000000.zip'
# 3inch 30% randomization
# path = 'models/3inch_drone/run2_3inch_30_percent/85000000.zip'
# 5inch fixed params
# path ='models/5inch_drone/run2_64_64_64/97000000.zip'
# 5inch 10% randomization
# path ='models/5inch_drone/run1_5inch_10_percent/63000000.zip'
# 5inch 20% randomization
# path ='models/5inch_drone/run1_5inch_20_percent/90000000.zip'
# 5inch 30% randomization
path ='models/5inch_drone/run0_5inch_30_percent/98000000.zip'

model = PPO.load(path)

# get network
network = list(model.policy.mlp_extractor.policy_net) + [model.policy.action_net]
network = nn.Sequential(*network)
print('NETWORK:')
print(network)

print(model.policy.action_dist)
print(model.policy.log_std)
print(model.policy.log_std.exp())
network_std = model.policy.log_std.exp().cpu().detach().numpy()
print(network_std)


NETWORK:
Sequential(
  (0): Linear(in_features=20, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=64, bias=True)
  (5): ReLU()
  (6): Linear(in_features=64, out_features=4, bias=True)
)
Parameter containing:
tensor([-1.9139, -1.9300, -1.9433, -1.9374], device='cuda:0',
       requires_grad=True)
tensor([0.1475, 0.1452, 0.1432, 0.1441], device='cuda:0',
       grad_fn=<ExpBackward0>)
[0.14750268 0.14515342 0.14322534 0.14407347]


In [23]:
from quad_race_env import *
from randomization import *
from quadcopter_animation import animation

#  ANIMATION FUNCTION
def animate_policy(model, env, **kwargs):
    env.reset()
    def run():
        actions, _ = model.predict(env.states)  
        states, rewards, dones, infos = env.step(actions)        
        return env.render()
    animation.view(run, gate_pos=env.gate_pos, gate_yaw=env.gate_yaw, **kwargs)

test_env = Quadcopter3DGates(num_envs=1, gates_pos=gate_pos, gate_yaw=gate_yaw, start_pos=start_pos, gates_ahead=1, randomization=randomization_fixed_params_5inch)

animate_policy(model, test_env)

In [24]:
import torch
import torch.nn as nn
import os
import subprocess

# remove the c_code folder and all of its contents
subprocess.call('rm -rf c_code', shell=True)
# create a new c_code folder
subprocess.call('mkdir c_code', shell=True)

# Create the "c_code" folder if it doesn't exist
output_folder = "c_code"
os.makedirs(output_folder, exist_ok=True)

# Generate the C file and the header file inside the "c_code" folder
source_file_path = os.path.join(output_folder, "neural_network.c")
header_file_path = os.path.join(output_folder, "neural_network.h")

# np.float32 to str
float_to_str = lambda x: str(float(x))

# Generate the C file
with open(source_file_path, "w") as file:
    file.write('#include "neural_network.h"\n')
    file.write("#include <stdio.h>\n")
    file.write("#include <math.h>\n\n")

    # Define weights and biases as global constant float arrays
    i = 1
    for layer in network:
        if isinstance(layer, nn.Linear):
            weights_layer = layer.weight.data.cpu().numpy()
            biases_layer = layer.bias.data.cpu().numpy()

            file.write(f"const float weights_fc{i}[] = {{\n")
            file.write(",\n".join([", ".join(map(float_to_str, row)) for row in weights_layer]))
            file.write("\n};\n\n")

            file.write(f"const float biases_fc{i}[] = {{\n")
            file.write(", ".join(map(float_to_str, biases_layer)))
            file.write("\n};\n\n")

            i+=1

    # LINEAR LAYER
    file.write("void nn_linear(const float* weights, const float* biases, const float* input, int in_features, int out_features, float* output) {\n")
    file.write("    for (int i = 0; i < out_features; ++i) {\n")
    file.write("        float neuron = biases[i];\n")
    file.write("        for (int j = 0; j < in_features; ++j) {\n")
    file.write("            neuron += input[j] * weights[i * in_features + j];\n")
    file.write("        }\n")
    file.write("        output[i] = neuron;\n")
    file.write("    }\n")
    file.write("}\n\n")

    # RELU LAYER
    file.write("void nn_relu(float* input, int size) {\n")
    file.write("    for (int i = 0; i < size; ++i) {\n")
    file.write("        input[i] = fmaxf(0, input[i]);\n")
    file.write("    }\n")
    file.write("}\n\n")

    # TANH LAYER
    file.write("void nn_tanh(float* input, int size) {\n")
    file.write("    for (int i = 0; i < size; ++i) {\n")
    file.write("        input[i] = tanh(input[i]);\n")
    file.write("    }\n")
    file.write("}\n\n")

    # FORWARD FUNCTION
    file.write("void nn_forward(const float* input, float* output) {\n")
    layer_size = network[0].out_features
    num_layers = sum(isinstance(layer, nn.Linear) for layer in network)
    i=0
    input_array = "input"
    for layer in network:
        if isinstance(layer, nn.Linear):
            i+=1
            if i<num_layers:
                file.write(f"    float fc{i}_output[{layer.out_features}];\n")
                file.write(f"    nn_linear(weights_fc{i}, biases_fc{i}, {input_array}, {layer.in_features}, {layer.out_features}, fc{i}_output);\n")
                input_array = f"fc{i}_output"
            else:
                file.write(f"    nn_linear(weights_fc{i}, biases_fc{i}, {input_array}, {layer.in_features}, {layer.out_features}, output);\n")
                input_array = "output"
            layer_size = layer.out_features
        elif isinstance(layer, nn.ReLU):
            file.write(f"    nn_relu({input_array}, {layer_size});\n")
        elif isinstance(layer, nn.Tanh):
            file.write(f"    nn_tanh({input_array}, {layer_size});\n")
        else:
            raise Exception(f"Unsupported layer: {layer}")
    file.write("}\n")

# Generate the header file
with open(header_file_path, "w") as header_file:
    header_file.write("#ifndef NEURAL_NETWORK_H\n")
    header_file.write("#define NEURAL_NETWORK_H\n\n")
    # Declare the forward function in the header file
    header_file.write("void nn_forward(const float* input, float* output);\n")
    header_file.write("\n#endif // NEURAL_NETWORK_H\n")


# Print the generated files
# Print the generated files
print(f"Generated {source_file_path}")
print(f"Generated {header_file_path}")

Generated c_code/neural_network.c
Generated c_code/neural_network.h


In [25]:
name = 'nn_controller'
# Create the "c_code" folder if it doesn't exist
output_folder = "c_code"
os.makedirs(output_folder, exist_ok=True)

# Generate the C file and the header file inside the "c_code" folder
source_file_path = os.path.join(output_folder, f"{name}.c")
header_file_path = os.path.join(output_folder, f"{name}.h")

num_gates = test_env.num_gates
gates_ahead = test_env.gates_ahead

# Generate the header file
with open(header_file_path, "w") as file:
    file.write(f"#ifndef {name.upper()}_H\n")
    file.write(f"#define {name.upper()}_H\n")
    file.write("\n")
    file.write("#include <stdint.h>\n")
    file.write("#include <stdbool.h>\n")
    file.write("\n")
    file.write(f'#define GATES_AHEAD {gates_ahead}\n')
    file.write(f'#define NUM_GATES {num_gates}\n')
    file.write("\n")
    # include neural network code
    file.write("// Include the neural network code\n")
    file.write("#include \"neural_network.h\"\n")
    file.write("\n")
    file.write("extern const float gate_pos[NUM_GATES][3];\n")
    file.write("extern const float gate_yaw[NUM_GATES];\n")
    file.write("extern const float start_pos[3];\n")
    file.write("extern const float start_yaw;\n")
    # file.write("const float gate_pos_rel[NUM_GATES][3];\n")
    # file.write("const float gate_yaw_rel[NUM_GATES];\n")
    file.write("extern uint8_t target_gate_index;\n")
    file.write("\n")
    # nn_reset function that resets the target gate index
    file.write("void nn_reset(void);\n")
    # nn_control function that that takes as input a float array of size 16 (world_state) and outputs an array of size 4 (rpms)
    
    #DISTURBANCE INPUT
    file.write("void nn_control(const float world_state[16], float motor_cmds[4]);\n")
    
    file.write("\n")
    file.write("#endif\n")

# Generate the C file
with open(source_file_path, "w") as file:
    file.write(f"#include \"{name}.h\"\n")
    file.write("#include <math.h>\n")
    file.write("#include <stdlib.h>\n")
    file.write("\n")
    # define boolean to set controller to determistic
    file.write("bool deterministic = false;\n")
    file.write("\n")
    file.write("const float output_std[4] = {\n")
    for i in range(4):
        file.write(f"    {network_std[i]},\n")
    file.write("};\n")
    file.write("\n")
    # define the gate positions and headings as const float arrays
    file.write("const float gate_pos[NUM_GATES][3] = {\n")
    for i in range(num_gates):
        file.write(f"    {{{test_env.gate_pos[i][0]}, {test_env.gate_pos[i][1]}, {test_env.gate_pos[i][2]}}},\n")
    file.write("};\n")
    file.write("\n")
    file.write("const float gate_yaw[NUM_GATES] = {\n")
    for i in range(num_gates):
        file.write(f"    {test_env.gate_yaw[i]},\n")
    file.write("};\n")
    file.write("\n")
    # define the start pos as a const float array
    file.write("const float start_pos[3] = {\n")
    file.write(f"    {test_env.start_pos[0]}, {test_env.start_pos[1]}, {test_env.start_pos[2]}\n")
    file.write("};\n")
    file.write("\n")
    # define start yaw equal to gate yaw[0]
    file.write(f"const float start_yaw = {test_env.gate_yaw[0]};\n")
    file.write("\n")
    # define the relative gate positions and headings as const float arrays
    file.write("const float gate_pos_rel[NUM_GATES][3] = {\n")
    for i in range(num_gates):
        file.write(f"    {{{test_env.gate_pos_rel[i][0]}, {test_env.gate_pos_rel[i][1]}, {test_env.gate_pos_rel[i][2]}}},\n")
    file.write("};\n")
    file.write("\n")
    file.write("const float gate_yaw_rel[NUM_GATES] = {\n")
    for i in range(num_gates):
        file.write(f"    {test_env.gate_yaw_rel[i]},\n")
    file.write("};\n")
    file.write("\n")
    # define the target gate index and set it to 0
    file.write("uint8_t target_gate_index = 0;\n")
    file.write("\n")
    file.write("void nn_reset(void) {\n")
    file.write("    target_gate_index = 0;\n")
    file.write("}\n")
    file.write("\n")
    file.write("void nn_control(const float world_state[16], float motor_cmds[4]) {\n")
    file.write("    // Get the current position, velocity and heading\n")
    file.write("    float pos[3] = {world_state[0], world_state[1], world_state[2]};\n")
    file.write("    float vel[3] = {world_state[3], world_state[4], world_state[5]};\n")
    file.write("    float yaw = world_state[8];\n")
    file.write("\n")
    file.write("    // Get the position and heading of the target gate\n")
    file.write("    float target_pos[3] = {gate_pos[target_gate_index][0], gate_pos[target_gate_index][1], gate_pos[target_gate_index][2]};\n")
    file.write("    float target_yaw = gate_yaw[target_gate_index];\n")
    file.write("\n")
    file.write("    // Set the target gate index to the next gate if we passed through the current one\n")
    file.write("    if (cosf(target_yaw) * (pos[0] - target_pos[0]) + sinf(target_yaw) * (pos[1] - target_pos[1]) > 0) {\n")
    file.write("        target_gate_index++;\n")
    file.write("        // loop back to the first gate if we reach the end\n")
    file.write("        target_gate_index = target_gate_index % NUM_GATES;\n")
    file.write("        // reset the target position and heading\n")
    file.write("        target_pos[0] = gate_pos[target_gate_index][0];\n")
    file.write("        target_pos[1] = gate_pos[target_gate_index][1];\n")
    file.write("        target_pos[2] = gate_pos[target_gate_index][2];\n")
    file.write("        target_yaw = gate_yaw[target_gate_index];\n")
    file.write("    }\n")
    file.write("\n")
    file.write("    // Get the position of the drone in gate frame\n")
    file.write("    float pos_rel[3] = {\n")
    file.write("        cosf(target_yaw) * (pos[0] - target_pos[0]) + sinf(target_yaw) * (pos[1] - target_pos[1]),\n")
    file.write("        -sinf(target_yaw) * (pos[0] - target_pos[0]) + cosf(target_yaw) * (pos[1] - target_pos[1]),\n")
    file.write("        pos[2] - target_pos[2]\n")
    file.write("    };\n")
    file.write("\n")
    file.write("    // Get the velocity of the drone in gate frame\n")
    file.write("    float vel_rel[3] = {\n")
    file.write("        cosf(target_yaw) * vel[0] + sinf(target_yaw) * vel[1],\n")
    file.write("        -sinf(target_yaw) * vel[0] + cosf(target_yaw) * vel[1],\n")
    file.write("        vel[2]\n")
    file.write("    };\n")
    file.write("\n")
    file.write("    // Get the heading of the drone in gate frame\n")
    file.write("    float yaw_rel = yaw - target_yaw;\n")
    file.write("    while (yaw_rel > M_PI) {yaw_rel -= 2*M_PI;}\n")
    file.write("    while (yaw_rel < -M_PI) {yaw_rel += 2*M_PI;}\n")
    file.write("\n")
    file.write("    // Get the neural network input\n")
    file.write("    float nn_input[16+4*GATES_AHEAD];\n")
    file.write("    // position and velocity\n")
    file.write("    for (int i = 0; i < 3; i++) {\n")
    file.write("        nn_input[i] = pos_rel[i];\n")
    file.write("        nn_input[i+3] = vel_rel[i];\n")
    file.write("    }\n")
    file.write("    // attitude\n")
    file.write("    nn_input[6] = world_state[6];\n")
    file.write("    nn_input[7] = world_state[7];\n")
    file.write("    nn_input[8] = yaw_rel;\n")
    file.write("    // body rates\n")
    file.write("    nn_input[9] = world_state[9];\n")
    file.write("    nn_input[10] = world_state[10];\n")
    file.write("    nn_input[11] = world_state[11];\n")
    file.write("    // motor rpms scaled to [-1,1]\n")
    file.write(f"    float w_min = {w_min_n};\n")
    file.write(f"    float w_max = {w_max_n};\n")
    file.write("    nn_input[12] = (world_state[12] - w_min) * 2 / (w_max - w_min) - 1;\n")
    file.write("    nn_input[13] = (world_state[13] - w_min) * 2 / (w_max - w_min) - 1;\n")
    file.write("    nn_input[14] = (world_state[14] - w_min) * 2 / (w_max - w_min) - 1;\n")
    file.write("    nn_input[15] = (world_state[15] - w_min) * 2 / (w_max - w_min) - 1;\n")
    file.write("\n")
    file.write("    // relative gate positions and headings\n")
    file.write("    for (int i = 0; i < GATES_AHEAD; i++) {\n")
    file.write("        uint8_t index = target_gate_index + i + 1;\n")
    file.write("        // loop back to the first gate if we reach the end\n")
    file.write("        index = index % NUM_GATES;\n")
    file.write("        nn_input[16+4*i]   = gate_pos_rel[index][0];\n")
    file.write("        nn_input[16+4*i+1] = gate_pos_rel[index][1];\n")
    file.write("        nn_input[16+4*i+2] = gate_pos_rel[index][2];\n")
    file.write("        nn_input[16+4*i+3] = gate_yaw_rel[index];\n")
    file.write("    }\n")
    file.write("    // Get the neural network output and write to the action array\n")
    file.write("    float nn_output[4];\n")
    file.write("    nn_forward(nn_input, nn_output);\n")
    file.write("\n")
    # if determinstic is false, add gaussian noise to the output
    file.write("    // add gaussian noise to the output\n")
    file.write("    if (!deterministic) {\n")
    file.write("        for (int i = 0; i < 4; i++) {\n")
    # generate random gaussian variables using the Box–Muller transform
    file.write("            // generate random gaussian variables using the Box–Muller transform\n")
    file.write("            float u1 = (float)rand() / RAND_MAX;\n")
    file.write("            float u2 = (float)rand() / RAND_MAX;\n")
    file.write("            float rand_std = sqrtf(-2 * logf(u1)) * cosf(2 * M_PI * u2);\n")
    file.write("            // add the noise to the output\n")
    file.write("            nn_output[i] += output_std[i] * rand_std;\n")
    file.write("        }\n")
    file.write("    }\n")
    file.write("\n")
    file.write("    for (int i = 0; i < 4; i++) {\n")
    motor_lim = test_env.motor_limit
    u_max = 2*motor_lim - 1
    file.write(f"        // clip the output to the range [-1, {u_max}] ({motor_lim} MOTOR LIMIT)\n")
    file.write(f"        if (nn_output[i] > {u_max}) nn_output[i] = {u_max};\n")
    file.write("        if (nn_output[i] < -1) {nn_output[i] = -1;}\n")
    file.write("        // map the output from [-1,1] to [0,1] \n")
    file.write("        motor_cmds[i] = (nn_output[i] + 1) / 2;\n")
    file.write("    }\n")
    file.write("}\n")

# Print the generated files
print(f"Generated {source_file_path}")
print(f"Generated {header_file_path}")

Generated c_code/nn_controller.c
Generated c_code/nn_controller.h


# Test C code

In [26]:
import os
import subprocess
import ctypes
import numpy as np
import importlib
importlib.reload(ctypes)

# https://cu7ious.medium.com/how-to-use-dynamic-libraries-in-c-46a0f9b98270
path = os.path.abspath('c_code')
# Create object files
subprocess.call('gcc -fPIC -c *.c', shell=True, cwd=path)
# Create library
subprocess.call('gcc -shared -Wl,-soname,libtools.so -o libtools.so *.o', shell=True, cwd=path)
# Remove object files
subprocess.call('rm *.o', shell=True, cwd=path)

lib_path = os.path.abspath("c_code/libtools.so")
fun = ctypes.CDLL(lib_path)

# define argument types 
fun.nn_forward.argtypes = [ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_float)]
fun.nn_control.argtypes = [ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_float)]

In [27]:
def c_network(x):
    x = np.array(x, dtype=np.float32)
    c_net_input = (ctypes.c_float*len(x))(*x)
    c_net_output = (ctypes.c_float*4)()
    fun.nn_forward(c_net_input, c_net_output)
    out = np.array(c_net_output[:])
    return np.clip(out, -1,1)

def torch_network(x):
    x = torch.tensor(x, dtype=torch.float32)
    out = network(x).cpu().detach().numpy()
    return np.clip(out, -1,1)

def nn_control(x):
    x = np.array(x, dtype=np.float32)

    # scale to [w_min, w_max]
    x[12:16] = (x[12:16] + 1)/2*(w_max_n - w_min_n) + w_min_n

    c_net_input = (ctypes.c_float*len(x))(*x)
    c_net_output = (ctypes.c_float*4)()
    
    
    fun.nn_control(c_net_input, c_net_output)
    out = np.array(c_net_output[:])
    # map back to [-1,1]
    out = (out*2) - 1
    return out


# test
x = np.random.rand(20)
full = lambda x: [float(xi) for xi in x]
print(full(c_network(x)))
print(full(torch_network(x)))
print(full(nn_control(x)))

[0.3130720853805542, 0.20864775776863098, 0.4531101882457733, -0.16590622067451477]
[-0.39422062039375305, 0.5065354108810425, -0.37285012006759644, 0.23397868871688843]
[-0.7303791642189026, 0.8453847169876099, -1.0, 1.0]


In [28]:
# Simulate C Network
test_env.reset()
fun.nn_reset()

crashes = []
steps = 0
action_list = []

def run():
    # state = test_env.states[0]
    # action1 = c_network(state.copy())
    # print(state)

    world_state = test_env.world_states[0]
    action = nn_control(world_state.copy())
    
    actions = np.array([action])
    action_list.append(action)

    steps = test_env.step_counts[0]+1
    states, rewards, dones, infos = test_env.step(actions)
    
    # print velocity
    print(np.linalg.norm(states[0, 3:6]))

    if dones[0]:
        crash = steps != test_env.max_steps
        crashes.append(crash)
        if crash:
            print('crash')
        else:
            print('success')
        fun.nn_reset()
    if len(crashes) == 100:
        print(f"Crash rate: {np.mean(crashes)}")
        # raise KeyboardInterrupt
    return test_env.render()

animation.view(run, gate_pos=test_env.gate_pos, gate_yaw=test_env.gate_yaw)

0.7443524
0.93678516
1.1048355
1.2524627
1.3723043
1.496098
1.6179518
1.7691606
1.9507309
2.1494458
2.3864615
2.6513355
2.933175
3.2398474
3.580639
3.9373305
4.30997
4.6874547
5.0648546
5.452411
5.856789
6.2809243
6.727641
7.1929264
7.672695
8.145688
8.598868
9.052553
9.491694
9.91975
10.338683
10.738934
11.13499
11.512907
11.86833
12.158391
12.364515
12.477508
12.509244
12.478681
12.41159
12.329176
12.2439575
12.161237
12.080236
11.99863
11.914721
11.831186
11.751083
11.681634
11.631985
11.610803
11.619783
11.65185
11.701485
11.757517
11.806199
11.835804
11.837555
11.806007
11.737536
11.637207
11.512121
11.37601
11.237116
11.107487
10.99672
10.905694
10.839869
10.804397
10.799128
10.816511
10.847801
10.885494
10.932691
10.99637
11.084091
11.195168
11.319563
11.453248
11.583723
11.699623
11.796224
11.870597
11.934352
11.987236
12.0369625
12.084115
12.13026
12.176933
12.216743
12.251695
12.280513
12.305443
12.33784
12.387026
12.461773
12.562495
12.691057
12.839543
12.984878
13.108052
13